## tf.data API를 사용한 더 나은 성능
##### url: https://www.tensorflow.org/guide/data_performance?hl=en#overview

- 목차
    - 개요
    - 자원
    - 설정
        - 데이터셋
        - 학습 루프
    - 성능 최적화
        - 간단한 접근법
        - 프리페칭
        - 데이터 추출 병렬화
        - 데이터 변환 병렬화
        - 캐싱
        - 매핑 함수 벡터화하기
        - 메모리 사용 흔적 줄이기
    - 모범 사례 요약
## 개요
GPU들과 TPU들은 단일 훈련 스탭을 수행하기 위한 소요 시간을 급격히 줄일 수 있습니다. <br>
최고 성능을 달성하는 것은 현재 스텝이 끝나기 전에 다음 스텝의 데이터를 운반하는 효율적인 입력 파이프라인이 요구됩니다.<br>
`tf.data` API는 유연하고, 효율적인 입력 파이프라인을 생성하는 것을 도와줍니다.<br>
해당 문서는 매우 성능이 좋은 텐서플로우 입력 파이프라인을 생성하기 위해 `tf.data` API를 어떻게 사용하는지 보여줍니다.<br><br>다음으로 넘어가기 전에 `tf.data` API를 어떻게 사용하는지 배울 수 있는 "[Build TensorFlot input pipelines](https://www.tensorflow.org/guide/data)" 가이드를 읽어보세요.

## 자원
- [Build TensorFlot input pipelines](https://www.tensorflow.org/guide/data)
- [`tf.data.Dataset'](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) API

## 설정

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import time

해당 가이드 전반에서, 반복적으로 데이터셋과 성능 측정을 할 예정입니다.<br>
다른 인자에 의해, 동일한 성능의 벤치마크를 만드는 것은 어려울 수 있습니다.

- 현재 CPU 로드율
- 네트워크 트래픽
- 캐시 같은 복잡한 매카니즘 등

동일한 성능의 벤치마크를 제공하기 위해 인공적으로 예제를 만들겠습니다.

### 데이터 셋
`tf.data.Dataset`를 상속 받은 `ArtificialDataset` 클래스를 정의합니다.<br>
해당 데이터셋은 다음과 같은 특징을 갖습니다.

- `num_samples` 만큼의 샘플을 생성합니다. (기본값 3)
- 파일을 열고 첫 번째 항목을 읽기 전에 잠시 대기합니다.
- 파일에서 데이터를 읽어와 각 아이템을 반환할 때마다 잠시 대기합니다.

In [2]:
class ArtificialDataset(tf.data.Dataset):
    def _generator(num_samples):
        # 파일 열기
        time.sleep(0.03)
        
        for sample_idx in range(num_samples):
            # 파일로부터 (line, record)로 이루어진 데이터를 읽습니다.
            time.sleep(0.015)
            
            yield (sample_idx,)
    
    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=tf.dtypes.int64,
            output_shapes=(1,),
            args=(num_samples,)
        )

이 데이터셋은 `tf.data.Datast.range`와 비슷하지만, 각 샘플의 시작과 사이에 고정된 딜레이가 추가되었습니다.

### 학습 루프
전반에 걸쳐 데이터셋을 반복하는 것이 얼마나 오래걸리는지 측정하기 위해 더미 학습 루프를 작성해야합니다.<br>
학습 시간을 시뮬레이팅 하겠습니다.

In [3]:
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # 학습을 수행 하는 단계
            time.sleep(0.01)
    tf.print('수행 시간: ', time.perf_counter() - start_time)

## 성능 최적화하기
어떻게 성능이 최적화될 수 있는지 보여주기 위해서, `ArtificialDataset`의 성능을 개선해야 합니다.

### 간단한 접근 방법
트릭을 사용하지 않고 간단한 파이프라인으로 시작해봅시다. 현재 데이터셋은 다음과 같이 반복됩니다.

In [4]:
benchmark(ArtificialDataset())

수행 시간:  0.2711333919999106


아래와 같이, 수행 시간이 어떻게 소비되었는지 볼 수 있습니다.

<img src=https://www.tensorflow.org/guide/images/data_performance/naive.svg />

학습 단계를 수행하는 것은 다음을 포함합니다. 
- 파일이 열리지 않았다면, 파일을 연다.
- 데이터 엔트리를 파일로부터 불러온다.
- 학습을 위해 데이터를 사용한다.

이와 같이 간단하게 동기화된 구현에서 모델은 파이프라인이 데이터를 파일로부터 불러오는 동안 유휴(idle) 상태에 있게 됩니다.<br>
반대로 모델이 학습하는 동안 입력 파이프라인은 유휴 상태에 있게 됩니다.<br>
이와 같이 학습 단계의 소요 시간은 파일 열기, 데이터 불러오기, 학습 시간의 합으로 이루어집니다.<br><br>
다음 섹션에서는 입력 파이프라인을 구축하고, 성능이 뛰어난 텐서플로우 입력 파이프라인 설계의 모범 사례를 보여줍니다.

### 프리페치(Prefetching)
프리페치는 학습 단계에서 모델 수행 및 전처리와 동시에 수앻됩니다. `s`번째 학습 단계에서 모델이 수행되는 동안, 입력 파이프라인은 `s+1`번째 데이터를 미리 읽습니다. <br>
이러한 방법으로 학습에 소요되는 시간과 데이터를 추출하기 위해 소요되는 시간을 최대한 줄일 수 있습니다.<br><br>
`tf.data` API는 `tf.data.Dataset.prefetch` 변환을 제공합니다. 해당 함수는 데이터를 소비하는 시간과 데이터를 생성하는 시간을 분리하는 작업에 사용될 수 있습니다.<br>
특히, 해당 함수는 백그라운드 스레드와 데이터가 요청되기 이전에 입력 데이터셋으로부터 원소를 프리페치하기 위한 내부 버퍼를 사용합니다.<br>
프리페치 될 원소의 수는 한번의 학습 단계에서 사용될 배치의 크기만큼이(또는 가능한 더 크게) 되어야 합니다.<br>
이러한 값을 수동으로 지정할 수 있거나 수행 중 동적으로 값을 튜닝하는 `tf.data` 런타임을 수행할 `tf.data.experimental.AUTOTUNE`으로 설정할 수 있습니다.<br><br>
프리페치 변환은 'Producer' 작업과 'Consumer' 작업이 겹칠 때마다 시간적 이익을 제공합니다.

In [5]:
benchmark(
    ArtificialDataset()
    .prefetch(tf.data.experimental.AUTOTUNE)
)

수행 시간:  0.21503757499999665


<img src=https://www.tensorflow.org/guide/images/data_performance/prefetched.svg />
샘플 0에 대한 학습이 수행되는 동안에 입력 파이프라인이 샘플 1의 데이터를 읽고 있는 것을 확인할 수 있습니다. 당연히 그 다음도 동일하게 이어집니다.

### 데이터 추출 병렬화
실제 현업에서는, 입력 데이터는 원격 저장소에 있을 확률이 높습니다.(ex, Google Cloud Storage 또는 HDFS 등)<br>
로컬 저장소에서 잘 작동하던 데이터 파이프라인일지라도 원격으로 데이터를 읽게되면 병목 현상이 발생할 수 있습니다. 이는 로컬 저장소와 원격 저장소의 차이 때문에 발생합니다.

- **Time-to-first-byte:** 원격 저장소에 저장된 파일의 첫 번째 바이트를 읽는 작업이 로컬 저장소보다 오래걸릴 수 있습니다.
- **Read throughput:** 원격 저장소는 일반적으로 넓은 대역폭을 제공합니다. 하지만, 파일 하나를 읽는 것은 그 대역폭의 작은 부분만 활용하게 됩니다.

추가적으로 원본 바이트를 메모리에 적재하게 되면, 추가적인 컴퓨팅 자원이 요구되는 데이터 해독 및 역 직렬화가 필요할 수 있습니다. (ex, protobuf) <br>
이러한 오버헤드는 데이터가 로컬 또는 원격에 저장되었는지 유무에 관계 없이 발생합니다. 하지만, 원격 저장소의 경우 프리페치가 효과적으로 이루어지지 않으면 더 나쁜 경우가 발생합니다.<br><br>
다양한 데이터 추출의 오버헤드를 완화하기 위해, `tf.data.Dataset.interleave` 변환은 데이터 로드 단계에서 다른 데이터셋의 내용을 인터리빙하는 병렬화로써 사용될 수 있습니다. <br>
겹쳐지는 데이터 셋의 수는 `cycle_length` 인자에 의해 명확히 정해질 수 있습니다. 또한, 병렬화 수준(level)은 `num_parallel_calls` 인자에 의해 정해질 수 있습니다. <br>
`prefetch` 변환과 비슷하게, `interleave` 변환은 `tf.data` 런타임에서 병렬화 수준 결정을 위임하는 `tf.data.experimental.AUTOTUNE`를 지원합니다.<br><br>
#### 순차적 인터리브
`tf.data.Dataset.interleave` 변환의 기본 인자는 두개의 데이터셋으로 부터 순차적으로 단일 인터리브 샘플을 생성합니다.

In [6]:
benchmark(
    tf.data.Dataset.range(2)
    .interleave(ArtificialDataset)
)

수행 시간:  0.22275559800004885


<img src=https://www.tensorflow.org/guide/images/data_performance/sequential_interleave.svg />
해당 플랏은 `interleave` 변환이 작동 방식을 보여줍니다. 2개의 사용 가능한 데이터셋으로부터 번갈아가며 샘플을 불러옵니다.<br>
하지만, 성능 향상이 일어나지는 않았습니다.<br><br>

#### 인터리브 병렬화

`interleave` 변환의 `num_parallel_calls` 인자를 사용해 보겠습니다. <br>
다수의 데이터셋을 병렬적으로 불러오며, 파일을 여는 것에 대한 대기 시간을 줄여줍니다.

In [7]:
benchmark(
    tf.data.Dataset.range(2)
    .interleave(
        ArtificialDataset,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
)

수행 시간:  0.14887254299992492


<img src=https://www.tensorflow.org/guide/images/data_performance/parallel_interleave.svg />
2개의 데이터셋이 병렬화 되어 읽어지고 있으며, 전역 데이터처리 시간이 감소되었습니다.

## 데이터 변환 병렬화
데이터를 준비할 때, 입력 원소들은 전처리를 필요로 할 수 있습니다. 이를 위해 `tf.data` API는 입력 데이터셋의 각 원소에 대해 사용자 정의 함수를 적용하는 `tf.data.Dataset.map` 변환을 제공합니다.<br>
입력 원소는 서로 독립적이기 때문에 전처리 연산은 다중 CPU 코어에 의해 병렬화 될 수 있습니다.<br>
이는 `prefetch` 및 `interleave`와 비슷하게 `map` 변환에서 제공하는 `num_parallel_calls` 인자를 통해 병렬화 수준을 지정할 수 있습니다.<br><br>
최적의 `num_parallel_calls` 인자의 값을 선택하는 것은, 사용자의 하드웨어와 학습 데이터의 특성 및 map 함수의 비용 그리고 같은 시간에 CPU에서 일어나는 연산들에 달려있습니다. <br>
간단하면서 자주 사용되는 값은 사용가능한 CPU 코어의 수로 지정됩니다.`prefetch`와 `interleave` 변환에서 그랬듯이 `map` 변환은 `tf.data` 런타임에서 병렬화 수준 결정을 위임하는 `tf.data.experimental.AUTOTUNE`를 지원합니다. 

In [8]:
def mapped_function(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s